In [1]:
from collections import defaultdict
from typing import Tuple

import imageio.v2 as imageio
import numpy as np
import numpy.typing as npt
import random
import os


In [2]:
# Calculate error
def root_mean_sq_err(ref_image, gen_image):
    m, n = gen_image.shape
    subtracted_image = gen_image.astype(float) - ref_image.astype(float)
    squared_image = np.square(subtracted_image)
    mean_image = squared_image / (n * m)
    err = np.sum(mean_image)

    return np.sqrt(err)


In [ ]:
def distance(
    query: npt.NDArray[np.float32], reference: npt.NDArray[np.float32]
) -> float:
    return np.linalg.norm(query - reference)


def find_closest_cluster(
    feature: npt.NDArray[np.float32], ids: npt.NDArray[np.float32]
) -> Tuple[int, npt.NDArray[np.float32]]:
    centr_ranks = [
        (idx, distance(feature, centroid)) for idx, centroid in enumerate(ids)
    ]
    centr_ranks.sort(key=lambda x: x[1])
    return centr_ranks[0]


In [ ]:
def k_means(
    image: npt.NDArray[np.float32],
    features: npt.NDArray[np.float32],
    k: int,
    n: int,
    M: int,
    N: int,
    seed: int,
):
    random.seed(seed)
    ids = np.sort(random.sample(range(0, M * N), k))
    clustered = defaultdict(list)
    for _ in range(n):
        clustered.clear()
        for feature in features:
            closest_idx, _ = find_closest_cluster(feature, ids)
            clustered[closest_idx].append(feature)
        for idx, feats in clustered.items():
            centroid = np.mean(feats)
            idx[idx] = centroid
    return clustered


In [3]:
def create_XY_features(M, N):
    X = np.tile(np.reshape(np.arange(M), (M, 1)), (1, N))
    Y = np.tile(np.reshape(np.arange(N), (N, 1)), (M, 1))
    X = np.reshape(X, (M * N, 1))
    Y = np.reshape(Y, (M * N, 1))
    XY = np.concatenate(X, Y, axis=1)
    return XY


def lum_reshape(image, M, N):
    return np.reshape(
        (0.299 * image[:, :, 0]) + (0.587 * image[:, :, 1]) + (0.114 * image[:, :, 2]),
        (M * N, 1),
    )


In [4]:
BASE_PATH = '../tests/resources/TestCases-InputImages'
image_filename = os.path.join(BASE_PATH, 'image_1.png')
ref_image_filename = os.path.join(BASE_PATH, 'image_1_ref1.png')
attributes_type = 1
k = 5
n = 10
seed = 42 #nice

image = imageio.imread(image_filename).astype(np.float32)
ref_image = imageio.imread(ref_image_filename).astype(np.float32)

M, N, _ = image.shape

match attributes_type:
    case 1:
        features = np.reshape(image, (M*N, 3))
    case 2:
        XY = create_XY_features(M, N)
        features = np.reshape(image, (M*N, 3))
        features = np.concatenate(features, XY, axis=1)
    case 3:
        features = lum_reshape(image, M, N)
    case 4:
        features = lum_reshape(image, M, N)
        features = np.concatenate(features, create_XY_features(M, N), axis=1)
    case _:
        exit(1)

clusters = k_means(image, features, k, n, seed)
